<a href="https://colab.research.google.com/github/rachocr/CCDEPLRL_EXERCISES/blob/main/Exercise8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 8

Instructions: Finetune a model to answer the questions below.

In [1]:
!pip install --upgrade transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.1/512.1 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 50.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.28.1
    Uninstalling huggingface-hub-0.28.1:
      Successfully uninstalled huggingface-hub-0.28.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers

In [2]:
pip install --upgrade transformers

In [3]:
!git config --global user.email "rachocr@gmail.com"
!git config --global user.name "rachocr"

In [4]:
from datasets import load_dataset

squad = load_dataset("squad", split="train[:5000]")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [5]:
squad = squad.train_test_split(test_size=0.2)

## A. Model finetuning

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [8]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [9]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [10]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [11]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-uncased")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForQuestionAnswering: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it

In [12]:

from transformers import create_optimizer

batch_size = 16
num_epochs = 2
total_train_steps = (len(tokenized_squad["train"]) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
)

In [13]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_squad["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_squad["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [14]:
import tensorflow as tf

model.compile(optimizer=optimizer)

In [15]:
from huggingface_hub import notebook_login
notebook_login()

In [17]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir="my_awesome_qa_model",
    tokenizer=tokenizer,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/naroo1/my_awesome_qa_model into local empty directory.


In [42]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=10, callbacks=[callback])

Epoch 1/10
250/250 [==============================] - 202s 807ms/step - loss: 1.5890 - val_loss: 1.8426
Epoch 2/10
250/250 [==============================] - 203s 813ms/step - loss: 1.5809 - val_loss: 1.8426
Epoch 3/10
250/250 [==============================] - 196s 785ms/step - loss: 1.5940 - val_loss: 1.8426
Epoch 4/10
250/250 [==============================] - 201s 803ms/step - loss: 1.5893 - val_loss: 1.8426
Epoch 5/10
250/250 [==============================] - 200s 802ms/step - loss: 1.5911 - val_loss: 1.8426
Epoch 6/10
250/250 [==============================] - 203s 814ms/step - loss: 1.5991 - val_loss: 1.8426
Epoch 7/10
250/250 [==============================] - 203s 812ms/step - loss: 1.5929 - val_loss: 1.8426
Epoch 8/10
250/250 [==============================] - 199s 798ms/step - loss: 1.5882 - val_loss: 1.8426
Epoch 9/10
250/250 [==============================] - 199s 796ms/step - loss: 1.5952 - val_loss: 1.8426
Epoch 10/10
250/250 [==============================] - 208s 833m

## B. Question Set

In [43]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="my_awesome_qa_model")

Some layers from the model checkpoint at my_awesome_qa_model were not used when initializing TFDistilBertForQuestionAnswering: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at my_awesome_qa_model and are newly initialized: ['dropout_59']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use 0


1. Question 1

In [44]:
context1 = "The Pacific Ocean is the largest and deepest of Earth's oceanic divisions. It extends from the Arctic Ocean in the north to the Southern Ocean in the south"
question1 = "What is the largest ocean on Earth?"
expectedAnswer1 = "Pacific Ocean"

2. Question 2

In [37]:
context2 = "Isaac Newton was an English mathematician, physicist, astronomer, and author who is widely recognised as one of the greatest mathematicians and physicists of all time"
question2 = "What was Isaac Newton's nationality?"
expectedAnswer2 = "English"

3. Question 3

In [38]:
context3 ="Mount Everest is Earth's highest mountain above sea level, located in the Himalayas on the border between Nepal and the Tibet Autonomous Region of China."
question3 = "where is Mount Everest located?"
expectedAnswer3 = "in the Himalayas on the border between Nepal and the Tibet Autonomous Region of China"

4. Question 4

In [39]:
context4 = "Photosynthesis is the process by which green plants and some other organisms use sunlight to synthesize foods from carbon dioxide and water"
question4 = "What do plants use to perform photosynthesis?"
expectedAnswer4 = "Sunlight"

5. Question 5

In [40]:
context5 = "Barack Obama served as the 44th president of the United States from 2009 to 2017."
question5 = "When did Barack Obama serve as president?"
expectedAnswer5 = "from 2009 to 2017"

In [45]:
questions = [question1, question2, question3, question4, question5]
contexts = [context1, context2, context3, context4, context5]
expected_answers = [expectedAnswer1, expectedAnswer2, expectedAnswer3, expectedAnswer4, expectedAnswer5]

for q, c, e in zip(questions, contexts, expected_answers):
    result = question_answerer(question=q, context=c)
    print(f"Context: {c}\nQuestion: {q}\nExpected Answer: {e}\nAnswer: {result['answer']}\n")


Context: The Pacific Ocean is the largest and deepest of Earth's oceanic divisions. It extends from the Arctic Ocean in the north to the Southern Ocean in the south
Question: What is the largest ocean on Earth?
Expected Answer: Pacific Ocean
Answer: Pacific Ocean

Context: Isaac Newton was an English mathematician, physicist, astronomer, and author who is widely recognised as one of the greatest mathematicians and physicists of all time
Question: What was Isaac Newton's nationality?
Expected Answer: English
Answer: English mathematician, physicist, astronomer

Context: Mount Everest is Earth's highest mountain above sea level, located in the Himalayas on the border between Nepal and the Tibet Autonomous Region of China.
Question: where is Mount Everest located?
Expected Answer: in the Himalayas on the border between Nepal and the Tibet Autonomous Region of China
Answer: Earth's highest mountain above sea level, located in the Himalayas

Context: Photosynthesis is the process by which g